# 0. 필요한 라이브러리 설치

In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/2025_캡스톤디자인/retention-ai/data/LGUPLUS_raw_data.csv", encoding='cp949')

# =======1. 고객 발언만 추출하는 함수 정의=======

In [4]:
def extract_customer_speech(content):
  customer_lines = re.findall(r'고객:\s*(.*?)(?=\n상담사:|\n고객:|$)', content, re.DOTALL)

  return " ".join(customer_lines).strip()

# =======2. 고객 발언 칼럼 생성=======

In [5]:
df['customer_content'] = df['consulting_content'].apply(extract_customer_speech)

# =======3. 마스킹 토큰 제거=======

In [6]:
patterns = r'<NAME>|<CHARGE>|<DATE>|<TIME>|<ADDRESS>|<PHONE_NUMBER>|<MOBILE_NUMBER>|<BIRTH_NUMBER>'

In [7]:
df['cleaned_content'] = df['customer_content'].str.replace(patterns, '', regex=True)
df['cleaned_content'] = df['cleaned_content'].str.replace(r'[^가-힣a-zA-Z0-9\s.,?!]', '', regex=True)

#=======4. consulting_turns 정규화=======

In [8]:
min_turns = df['consulting_turns'].min()
max_turns = df['consulting_turns'].max()
df['normalized_turns'] = (df['consulting_turns'] - min_turns) / (max_turns - min_turns)

### 상담 카테고리 별 심각도 가중치 부여

In [9]:
def get_weight(category):
  # 고위험
  if '해지' in category or '요금' in category or '청구' in category:
    return 0.8
  # 중위험
  elif '서비스' in category:
    return 0.6
  # 단순 문의
  else:
    return 0.3

In [10]:
df['category_weight'] = df['consulting_category'].apply(get_weight)

In [11]:
df.head()

,source_id,source,consulting_date,consulting_category,client_gender,client_age,consulting_turns,consulting_length,consulting_content,customer_content,cleaned_content,normalized_turns,category_weight
0,43118,엘지유플러스,NaN,요금제 변경,NaN,NaN,63,395,"상담사: 일상의 즐거운 변화 LGU+ <NAME>입니다.\n고객: 아 네, 안녕하세...","아 네, 안녕하세요? 네, 요금제 관련된 문의 드릴려고 하는데요. 아 우선 지금 현...","아 네, 안녕하세요? 네, 요금제 관련된 문의 드릴려고 하는데요. 아 우선 지금 현...",0.525773,0.8
1,42734,엘지유플러스,NaN,부가서비스 안내,NaN,NaN,46,310,"상담사: 일상의 즐거운 변화 LGU+ <NAME>입니다.\n고객: 예, 안녕하세요?...","예, 안녕하세요? 제가 그 뭐야 보이스피싱에 연류돼가지고 그 뭐야 명의도용 가입제한...","예, 안녕하세요? 제가 그 뭐야 보이스피싱에 연류돼가지고 그 뭐야 명의도용 가입제한...",0.350515,0.6
2,41107,엘지유플러스,NaN,소액 결제,NaN,NaN,32,243,"상담사: 일상의 즐거운 변화 LGU+ <NAME>입니다.\n고객: 예, 소액결제한도...","예, 소액결제한도를 늘렸는데요. <DATE>이 미납 그 기록이 남아서 결제가 안된다...","예, 소액결제한도를 늘렸는데요. 이 미납 그 기록이 남아서 결제가 안된다고 하는데 ...",0.206186,0.3
3,42440,엘지유플러스,NaN,요금 납부,NaN,NaN,38,403,상담사: 일상의 즐거운 변화 LGU+ <NAME>입니다.\n고객: 여보세요?\n상담...,"여보세요? 예, 요금이 좀 미납되어서 요금 좀 낼려고 하는데요. 예. 가상계좌로 좀...","여보세요? 예, 요금이 좀 미납되어서 요금 좀 낼려고 하는데요. 예. 가상계좌로 좀...",0.268041,0.8
4,43046,엘지유플러스,NaN,선택약정 할인,NaN,NaN,39,286,상담사: 일상의 즐거운 변화 LGU+ <NAME>입니다.\n고객: 약정할인 좀 받고...,"약정할인 좀 받고 싶어서요. 네. 1년 해주세요. 예. 예. 네. 네. 네. 네, ...","약정할인 좀 받고 싶어서요. 네. 1년 해주세요. 예. 예. 네. 네. 네. 네, ...",0.278351,0.3


# =======4. 불필요한 칼럼 제거=======

In [13]:
columns_to_drop = [
    'source',
    'consulting_content',
    'customer_content',
    'consulting_length',
    'consulting_date',
    'client_gender',
    'client_age',
]

df = df.drop(columns=columns_to_drop, errors='ignore')

In [14]:
print(df.columns.to_list())

['source_id', 'consulting_category', 'consulting_turns', 'cleaned_content', 'normalized_turns', 'category_weight']


In [18]:
df['consulting_category'].unique()

array(['요금제 변경', '부가서비스 안내', '소액 결제', '요금 납부', '선택약정 할인', '결합할인',
       '인터넷 장애/고장', '명의/번호/유심 해지', '납부확인서', '통화품질', '납부 방법 변경', '요금 안내',
       '가입 안내', '휴대폰 정지/분실/파손', '기기변경', '내역 조회', '요금 할인', '군입대 일시정지',
       '보험 안내', '복지할인 등록', '장비 안내', '보험 청구', '인터넷 속도/품질', '상품 안내', '해외로밍',
       '위약금 안내', '개통점 안내', '세금계산서', '청구서', '기기할부 약정', '청구지 변경',
       '명의/번호/유심 일시정지', '이전 설치', '정보변경', '명의/번호/유심 변경', '배송문의',
       '인터넷 가입 및 변경', '번호 변경', '데이터 옵션 안내', '멤버십 혜택', '파손보험 안내', '번호이동',
       '해지 안내', '보험 해지', '인터넷전화가입 및 변경', '장기고객 혜택', '환불 안내', '한도 안내',
       '보험 혜택', '번호 이동', '기타 할인 혜택', '제휴카드로 할인', '반품 안내',
       '명 의/번호/유심 일시정지', '카드 배송', '약정 안내', '보험 신청', '자동 이체', '납부 확인서'],
      dtype=object)

#=======5. 최종 데이터 저장=======

In [19]:
df.to_csv('lg_cleaned_data.csv', index=False)